# 0.**Librerias**




Colab T4



In [ ]:
!pip install -U torch --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers accelerate datasets bitsandbytes triton peft trl \
    bert-score textstat numpy pandas
# ===============================
# ✅ LIMPIEZA DE SALIDA Y VERIFICACIÓN DE VERSIONES
# ===============================
import IPython
IPython.display.clear_output(wait=True)



Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 100.9 M

In [ ]:
BASE_MODEL = "meta-llama/Llama-3.2-3B"  # o el modelo base que usaste originalmente
ADAPTER_PATH = "/content/drive/MyDrive/llama3_models/adapter_v6_L4_6_epochs_CE_KL_2000_NO_instruct_Hi_v2"
ref = "PLS_v6_L4_6_epochs_CE_KL_2000_NO_instruct_Hi_v2"

In [ ]:
#!pip install --upgrade transformers accelerate bitsandbytes

In [ ]:
import torch, transformers, peft, trl, accelerate, bitsandbytes, datasets

print("✅ Librerías cargadas correctamente:\n")
print(f"PyTorch:        {torch.__version__}")
print(f"Transformers:   {transformers.__version__}")
print(f"PEFT:           {peft.__version__}")
print(f"TRL:            {trl.__version__}")
print(f"Accelerate:     {accelerate.__version__}")
print(f"Datasets:       {datasets.__version__}")
print(f"BitsAndBytes:   {bitsandbytes.__version__}")

print("\n✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨")

✅ Librerías cargadas correctamente:

PyTorch:        2.9.0+cu126
Transformers:   4.57.1
PEFT:           0.18.0
TRL:            0.25.1
Accelerate:     1.12.0
Datasets:       4.4.1
BitsAndBytes:   0.48.2

✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Crear carpeta y entrar
!mkdir cochrane_data
%cd cochrane_data

# 2. Inicializar git vacío
!git init

# 3. Agregar remote
!git remote add -f origin https://github.com/feliperussi/bridging-the-gap-in-health-literacy.git

# 4. Activar modo sparse checkout
!git config core.sparseCheckout true

# 5. Indicar SOLO las carpetas que queremos
!echo "data_collection_and_processing/Data Sources/Cochrane/train/pls" >> .git/info/sparse-checkout
!echo "data_collection_and_processing/Data Sources/Cochrane/train/non_pls" >> .git/info/sparse-checkout

# 6. Descargar solo esa parte del repo
!git pull origin main


mkdir: cannot create directory ‘cochrane_data’: File exists
/content/cochrane_data
Reinitialized existing Git repository in /content/cochrane_data/.git/
error: remote origin already exists.
From https://github.com/feliperussi/bridging-the-gap-in-health-literacy
 * branch                main       -> FETCH_HEAD
Already up to date.


Gestionar Hugging Face

In [ ]:
import os
os.environ["HF_TOKEN"] = "hf_"

In [ ]:
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
import glob, pandas as pd
import transformers, trl, peft, accelerate, datasets
import bitsandbytes as bnb
import re

# 1.Construir el dataset non_pls → pls desde las carpetas de GitHub

In [ ]:
import re

# Número máximo de pares
pares = 2000

# === 1️⃣ Listar archivos ===
pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/pls/*.txt"))
non_pls_files = sorted(glob.glob("data_collection_and_processing/Data Sources/Cochrane/train/non_pls/*.txt"))

# === 2️⃣ Filtrar archivos que contengan 'accumulated' en AMBOS ---
pls_files = [f for f in pls_files if "accumulated" not in os.path.basename(f).lower()]
non_pls_files = [f for f in non_pls_files if "accumulated" not in os.path.basename(f).lower()]

print(f"📚 Total PLS (sin accumulated): {len(pls_files)}")
print(f"📂 Total non-PLS (sin accumulated): {len(non_pls_files)}")

# === 3️⃣ Función para extraer el ID base ===
def extract_id(filename):
    base = os.path.basename(filename)
    match = re.match(r"(.+?)\.pub\d+", base)
    return match.group(1) if match else base.split(".txt")[0]

# === 4️⃣ Diccionarios {id_base: ruta} ===
pls_dict = {extract_id(p): p for p in pls_files}
non_pls_dict = {extract_id(n): n for n in non_pls_files}

# === 5️⃣ Emparejar ===
data = []
for base_id, non_path in non_pls_dict.items():
    if base_id in pls_dict:
        pls_path = pls_dict[base_id]
        with open(pls_path, "r", encoding="utf-8") as f1, open(non_path, "r", encoding="utf-8") as f2:
            data.append({
                "id": base_id,
                "file_pls": os.path.basename(pls_path),
                "file_non_pls": os.path.basename(non_path),
                "pls": f1.read().strip(),
                "non_pls": f2.read().strip()
            })
    if len(data) >= pares:
        break

# === 6️⃣ DataFrame ===
df = pd.DataFrame(data)

# === 7️⃣ Guardar ===
output_path = "cochrane_pairs_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Emparejados {len(df)} pares por nombre base (sin 'accumulated' en ninguno)")
print(f"💾 Guardado en: {output_path}")
df.head(3)



📚 Total PLS (sin accumulated): 4797
📂 Total non-PLS (sin accumulated): 7251

✅ Emparejados 2000 pares por nombre base (sin 'accumulated' en ninguno)
💾 Guardado en: cochrane_pairs_clean.csv


,id,file_pls,file_non_pls,pls,non_pls
0,10.1002-14651858.CD000006,10.1002-14651858.CD000006.pub2-pls.txt,10.1002-14651858.CD000006.pub2-abstract.txt,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...
1,10.1002-14651858.CD000009,10.1002-14651858.CD000009.pub4-pls.txt,10.1002-14651858.CD000009.pub4-abstract.txt_se...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...
2,10.1002-14651858.CD000012,10.1002-14651858.CD000012.pub4-pls.txt,10.1002-14651858.CD000012.pub4-abstract.txt_se...,Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl..."


Cargar desde archivos

In [ ]:
df = df.rename(columns={"non_pls": "source", "pls": "target"})

INSTR_PROMPT = """Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description of the design,
description of study and patient population (age, health condition, gender), and the expected
amount of time a person will be in the study.
4. Results: Answer ‘What were the main results of the study’, include the benefits for the patients,
how the study was relevant for the area of study, and the conclusions from the investigator.
c. Consistency and Replicability: The generated PLS should be consistent regardless of the order of
sentences or the specific phrasing used in the input protocol text.
d. Compliance with Plain Language Guidelines: The generated PLS must follow all these plain
language guidelines:
• Have readability grade level of 6 or below.
• Do not have jargon. All technical or medical words or terms should be defined or broken down
into simple and logical explanations.
• Active voice, not passive.
• Mostly one or two syllable words.
• Sentences of 15 words or less.
• Short paragraphs of 3-5 sentences.
• Simple numbers (e.g., ratios, no percentages).
e. Do not invent Content: The AI model should not invent information. If the AI model includes data
other than the one given in the input abstract, the AI model should guarantee such data is verified and
real.
f. Aim for an approximate PLS length of 500-900 words.

Input abstract:

{source}

Output PLS:
"""

def format_example(row):
    prompt = INSTR_PROMPT.format(source=row["source"])
    return {"prompt": prompt, "response": row["target"]}

# Aplicar la función a cada fila del DataFrame
dataset = df.apply(format_example, axis=1).to_list()

# Crear Dataset de Hugging Face y dividir en train/test
from datasets import Dataset

hf_ds = Dataset.from_list(dataset).train_test_split(test_size=0.1, seed=42)

print(hf_ds)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 200
    })
})


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq, BitsAndBytesConfig, AutoModel,TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import os
import sys
import torch
from torch import nn
from transformers import Trainer
import textstat
import pyphen
import re




In [ ]:
print(f"Dataset original size:")
print(f"   Train: {len(hf_ds['train'])}")
print(f"   Test: {len(hf_ds['test'])}")

Dataset original size:
   Train: 1800
   Test: 200


# Recuperar modelo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [ ]:
#BASE_MODEL = "meta-llama/Llama-3.2-3B"  # o el modelo base que usaste originalmente
#ADAPTER_PATH = "/content/drive/MyDrive/llama3_models/adapter_v6_L4_6_epochs_CE_KL_2000_NO_instruct_Hi_v2"

# 1. Cargar modelo base
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype="auto",
    device_map="auto"
)

# 2. Aplicar el LoRA adapter
from peft import PeftModel
model = PeftModel.from_pretrained(model, ADAPTER_PATH)

# 3. Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print("✅ Modelo y tokenizer cargados con el adapter LoRA aplicado.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

✅ Modelo y tokenizer cargados con el adapter LoRA aplicado.


# 2.Generación de Resumenes


## Preparar Dataset para pasar a DataFrame df_test para la Generación de Texto

In [ ]:
print(hf_ds["test"].column_names)
df_test = hf_ds["test"].to_pandas()
print(df_test.columns)

import re
import pandas as pd

def extract_source_from_prompt(prompt):
    """
    Extrae el texto entre 'original text' y 'simplified text' del prompt.
    Robusto ante espacios, mayúsculas y saltos de línea.
    """
    match = re.search(
        r"Input abstract:\s*[:\-–]?\s*(.*?)\s*Output PLS",
        prompt,
        re.IGNORECASE | re.DOTALL
    )
    if match:
        return match.group(1).strip()
    return ""

# Crear columna 'source' desde 'prompt'
df_test["source"] = df_test["prompt"].apply(extract_source_from_prompt)

# Mantener solo columnas necesarias
df_test = df_test[["source", "response"]]

# Mostrar ejemplo de verificación
print(df_test.head(2))

['prompt', 'response']
Index(['prompt', 'response'], dtype='object')
                                              source  \
0  Background\nPeriodic fever, aphthous stomatiti...   
1  Background\nMemory problems are a common cogni...   

                                            response  
0  Tonsillectomy for PFAPA syndrome (a rare cause...  
1  Cognitive rehabilitation for memory deficits a...  


## GENERACION

In [ ]:
if "trainer" in locals() and hasattr(trainer, "model"):
    fine_tuned_model = trainer.model.eval()
else:
    fine_tuned_model = model.eval()

In [ ]:
import gc
import torch
from tqdm import tqdm
from difflib import SequenceMatcher
import re

def generate_simplifications_v2(
    df,
    model,
    tokenizer,
    instr_prompt,
    batch_size=2,
    max_source_tokens=512,
    sim_threshold=0.75,
    verbose=True,
):
    """
    Genera textos simplificados con regeneración multinivel.
    Si el texto generado está vacío, intenta regenerar automáticamente.
    """

    import gc, torch, re
    from tqdm import tqdm
    from difflib import SequenceMatcher

    tokenizer.padding_side = "right"
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.clean_up_tokenization_spaces = True

    model.eval()
    model.config.use_cache = True
    model.config.output_hidden_states = False
    model.config.return_dict = True

    torch.cuda.empty_cache()
    gc.collect()

    if verbose:
        print("\n🚀 Generación iniciada (modo seguro, con detección de vacíos)...")

    # === FUNCIONES AUXILIARES ===
    def truncate_text(text, max_tokens=max_source_tokens):
        tokens = text.split()
        if len(tokens) <= max_tokens:
            return text
        head = tokens[:max_tokens // 2]
        tail = tokens[-max_tokens // 2:]
        return " ".join(head + ["[...]"] + tail)

    def safe_generate(prompts, gen_params):
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1536
        ).to(model.device)

        with torch.inference_mode():
            outputs = model.generate(**inputs, **gen_params)

        texts = tokenizer.batch_decode(
            outputs,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            errors="replace"
        )
        return texts

    def extract_simplified_text(text):
        match = re.search(r"Output PLS\s*[:\-–]?\s*(.*)", text, re.IGNORECASE | re.DOTALL)
        if match:
            simplified = match.group(1).strip()
        else:
            simplified = text.strip()
        simplified = re.sub(r"<\|endoftext\|>", "", simplified)
        simplified = re.sub(r"\s+", " ", simplified).strip()
        return simplified

    # === PARÁMETROS DE GENERACIÓN ===
    GEN_PARAMS_L1 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 1, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L2 = {"max_new_tokens": 256, "temperature": 1.1, "top_p": 0.85, "do_sample": True,
                     "num_beams": 1, "repetition_penalty": 1.0,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L3 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 4, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    # === PREPARACIÓN ===
    df = df.copy()
    df["source_for_generation"] = df["source"].apply(lambda x: truncate_text(x))
    if verbose:
        print("✅ Columna 'source_for_generation' creada correctamente")

    # === GENERACIÓN PRINCIPAL ===
    generated_texts, is_copy_list, was_regenerated_list = [], [], []
    regeneration_level_list, similarity_scores = [], []
    regeneration_count = 0

    for i in tqdm(range(0, len(df), batch_size), desc="Generando simplificaciones"):
        if i % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()

        batch_sources = df["source"].iloc[i:i+batch_size].tolist()
        batch_sources_trunc = df["source_for_generation"].iloc[i:i+batch_size].tolist()
        batch_prompts = [instr_prompt.format(source=s) for s in batch_sources_trunc]

        full_texts = safe_generate(batch_prompts, GEN_PARAMS_L1)

        for j, full_text in enumerate(full_texts):
            simplified = extract_simplified_text(full_text)
            source_idx = i + j
            if source_idx >= len(df):
                continue

            # 🔍 --- NUEVO BLOQUE: REGENERACIÓN SI EL TEXTO ESTÁ VACÍO ---
            if not simplified or len(simplified.strip()) == 0:
                if verbose:
                    print(f"⚠️ Texto vacío detectado en índice {source_idx}. Regenerando...")
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])
                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified = extract_simplified_text(regen_text)
                # Si sigue vacío, prueba con L3
                if not simplified or len(simplified.strip()) == 0:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified = extract_simplified_text(beam_text)

            # === Calcular similitud y regeneraciones ===
            sim = SequenceMatcher(None, batch_sources[j].lower(), simplified.lower()).ratio()
            is_copy = sim > sim_threshold
            regen_level = 0

            if is_copy:
                regeneration_count += 1
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])

                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified_regen = extract_simplified_text(regen_text)
                sim_regen = SequenceMatcher(None, batch_sources[j].lower(), simplified_regen.lower()).ratio()

                if sim_regen <= sim_threshold:
                    simplified, sim, is_copy, regen_level = simplified_regen, sim_regen, False, 2
                else:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified_beam = extract_simplified_text(beam_text)
                    sim_beam = SequenceMatcher(None, batch_sources[j].lower(), simplified_beam.lower()).ratio()

                    if sim_beam <= sim_threshold:
                        simplified, sim, is_copy, regen_level = simplified_beam, sim_beam, False, 3
                    else:
                        if sim_beam < sim_regen:
                            simplified, sim, regen_level = simplified_beam, sim_beam, 3
                        else:
                            simplified, sim, regen_level = simplified_regen, sim_regen, 2

            generated_texts.append(simplified)
            is_copy_list.append(is_copy)
            was_regenerated_list.append(regen_level > 0)
            regeneration_level_list.append(regen_level)
            similarity_scores.append(sim)

    # === GUARDAR RESULTADOS ===
    df["generated_text"] = generated_texts
    df["is_copy"] = is_copy_list
    df["was_regenerated"] = was_regenerated_list
    df["regeneration_level"] = regeneration_level_list
    df["similarity_score"] = similarity_scores

    if verbose:
        print(f"\n✅ Generación completada sin errores de memoria")
        print(f"📊 Total textos: {len(df)} | Regenerados: {sum(was_regenerated_list)} | Copias finales: {sum(is_copy_list)}")
        print(f"Promedio de similitud: {sum(similarity_scores)/len(similarity_scores):.3f}")

    return df


## Necesita source y response

In [ ]:
df_test_50 = df_test.sample(n=50, random_state=42).reset_index(drop=True)

In [ ]:
df_test_50

,source,response
0,Background\nThis updated Cochrane Review of re...,Reminiscence therapy for dementia\nReview ques...
1,Background\nIn many countries emergency depart...,Primary care professionals providing non‐urgen...
2,Background\nPsychotherapy is regarded as the f...,Psychological therapies for borderline persona...
3,Background\nSpina bifida is a fetal neural tub...,Spina bifida repair and infant and maternal he...
4,We included 12 trials in the original review (...,Transcutaneous Electrical Nerve Stimulation (T...
5,Background\nCocaine dependence is a major publ...,Anticonvulsants for cocaine dependence\nBackgr...
6,Background\nAcute respiratory distress syndrom...,Positioning for hospitalised infants and child...
7,Background\nUterine fibroids occur in up to 40...,Preoperative medical therapy before surgery fo...
8,Background\nAcute exacerbations of chronic obs...,Do systemic corticosteroids improve treatment ...
9,"Background\nIn people with haemophilia, therap...",Recombinant (non‐human) factor VIIa clotting f...


In [ ]:
df_out = generate_simplifications_v2(
    df=df_test_50,
    model=fine_tuned_model,
    tokenizer=tokenizer,
    instr_prompt=INSTR_PROMPT,
    batch_size=2,
)


🚀 Generación iniciada (modo seguro, con detección de vacíos)...
✅ Columna 'source_for_generation' creada correctamente


Generando simplificaciones:   0%|          | 0/25 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Generando simplificaciones: 100%|██████████| 25/25 [27:24<00:00, 65.79s/it]


✅ Generación completada sin errores de memoria
📊 Total textos: 50 | Regenerados: 0 | Copias finales: 0
Promedio de similitud: 0.040


In [ ]:
#ref = "PLS_v6_L4_6_epochs_CE_KL_2000_NO_instruct_Hi_v2"
filename = f"pls_results_{ref}.csv"
save_path = f"/content/drive/MyDrive/{filename}"

df_out.to_csv(save_path, index=False, encoding="utf-8")
print(f"📁 Archivo guardado en: {save_path}")

📁 Archivo guardado en: /content/drive/MyDrive/pls_results_PLS_v6_L4_6_epochs_CE_KL_2000_NO_instruct_Hi_v2.csv


In [ ]:
# =====================================================================
# EJEMPLO DE SALIDA SIMPLIFICADA
# =====================================================================
print("\n🧾 Ejemplo de salida simplificada:\n" + "="*50)

# Seleccionamos un ejemplo aleatorio del DataFrame
import random
idx = random.randint(0, len(df_out) - 1)

print(f"\n🔹 Índice: {idx}")
print("\n--- Texto original (source) ---")
print(df_out.loc[idx, "source"][:1000])  # mostramos hasta 1000 caracteres por legibilidad

print("\n--- Texto simplificado generado ---")
print(df_out.loc[idx, "generated_text"])

print("\n" + "="*50)


🧾 Ejemplo de salida simplificada:

🔹 Índice: 38

--- Texto original (source) ---
Background
Mechanical ventilation is a critical component of paediatric intensive care therapy. It is indicated when the patient’s spontaneous ventilation is inadequate to sustain life. Weaning is the gradual reduction of ventilatory support and the transfer of respiratory control back to the patient. Weaning may represent a large proportion of the ventilatory period. Prolonged ventilation is associated with significant morbidity, hospital cost, psychosocial and physical risks to the child and even death. Timely and effective weaning may reduce the duration of mechanical ventilation and may reduce the morbidity and mortality associated with prolonged ventilation. However, no consensus has been reached on criteria that can be used to identify when patients are ready to wean or the best way to achieve it. 
Objectives
To assess the effects of weaning by protocol on invasively ventilated critically ill childr

Algunos modelos de instrucciones (como Alpaca, Mistral, etc.) incluyen el prompt dentro del output.
Puedes limpiarlo fácilmente:

In [ ]:
'''import re

def extract_response(gen):
    """
    Extrae la parte del texto posterior a 'Simplified text:' (en cualquiera de sus variantes)
    y limpia encabezados residuales tipo '## Background' o 'Output:'.
    """
    # Captura flexible: acepta ###, espacios, o saltos de línea
    match = re.search(r"#+\s*Output PLS\s*[:\-–]?\s*(.*)", gen, flags=re.IGNORECASE | re.DOTALL)
    if match:
        response = match.group(1).strip()
    else:
        response = gen.strip()

    # Limpieza opcional (cabeceras Markdown o Output:)
    response = re.sub(r"^(Output:|#+\s*(Background|Summary|Question|Answer).*)", "", response, flags=re.IGNORECASE).strip()
    return response

df_out["generated_clean"] = df_out["generated_text"].apply(extract_response)



SyntaxError: incomplete input (ipython-input-2631458018.py, line 1)

In [ ]:
df_out.head()

,source,response,source_for_generation,generated_text,is_copy,was_regenerated,regeneration_level,similarity_score
0,Background\nThis updated Cochrane Review of re...,Reminiscence therapy for dementia\nReview ques...,Background This updated Cochrane Review of rem...,**Plain Title** Revisiting Reminiscence Therap...,False,False,0,0.010
1,Background\nIn many countries emergency depart...,Primary care professionals providing non‐urgen...,Background In many countries emergency departm...,**Plain Title** Reducing Overcrowding in Emerg...,False,False,0,0.060
2,Background\nPsychotherapy is regarded as the f...,Psychological therapies for borderline persona...,Background Psychotherapy is regarded as the fi...,**Plain Title** Borderline Personality Disorde...,False,False,0,0.008
3,Background\nSpina bifida is a fetal neural tub...,Spina bifida repair and infant and maternal he...,Background Spina bifida is a fetal neural tube...,**Plain Title** Spina Bifida Repair: Does Earl...,False,False,0,0.013
4,We included 12 trials in the original review (...,Transcutaneous Electrical Nerve Stimulation (T...,We included 12 trials in the original review (...,**Plain Title** TENS Therapy for Acute Pain Re...,False,False,0,0.044



# 3.Evaluación y Métricas


## Librerias de Metricas

In [ ]:
# Verificar
print(df_test["source"].head(1).values[0])

Background
Periodic fever, aphthous stomatitis, pharyngitis and cervical adenitis (PFAPA) syndrome is a rare clinical syndrome of unknown cause usually identified in children. Tonsillectomy is considered a potential treatment option for this syndrome. This is an update of a Cochrane Review first published in 2010 and previously updated in 2014. 
Objectives
To assess the effectiveness and safety of tonsillectomy (with or without adenoidectomy) compared with non‐surgical treatment in the management of children with PFAPA. 
Search methods
The Cochrane ENT Information Specialist searched the Cochrane ENT Trials Register; Central Register of Controlled Trials (CENTRAL 2019, Issue 4); PubMed; Ovid Embase; CINAHL; Web of Science; ClinicalTrials.gov; ICTRP and additional sources for published and unpublished trials. The date of the search was 15 October 2019. 
Selection criteria
Randomised controlled trials comparing tonsillectomy (with or without adenoidectomy) with non‐surgical treatment in 

## Todas las métricas

In [ ]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
print(df_out.columns)

Index(['source', 'response', 'source_for_generation', 'generated_text', 'is_copy', 'was_regenerated',
       'regeneration_level', 'similarity_score'],
      dtype='object')


In [ ]:
df_out.head()

,source,response,source_for_generation,generated_text,is_copy,was_regenerated,regeneration_level,similarity_score
0,Background\nThis updated Cochrane Review of re...,Reminiscence therapy for dementia\nReview ques...,Background This updated Cochrane Review of rem...,defining dementia Dementia is a brain disorder...,False,False,0,0.006771
1,Background\nIn many countries emergency depart...,Primary care professionals providing non‐urgen...,Background In many countries emergency departm...,Plain Title: Primary Care Professionals in Hos...,False,False,0,0.063577
2,Background\nPsychotherapy is regarded as the f...,Psychological therapies for borderline persona...,Background Psychotherapy is regarded as the fi...,Plain Title: Borderline Personality Disorder T...,False,False,0,0.015605
3,Background\nSpina bifida is a fetal neural tub...,Spina bifida repair and infant and maternal he...,Background Spina bifida is a fetal neural tube...,defining the study Spina bifida is a serious c...,False,False,0,0.033030
4,We included 12 trials in the original review (...,Transcutaneous Electrical Nerve Stimulation (T...,We included 12 trials in the original review (...,defining the effect of transcutaneous electric...,False,False,0,0.057851


In [ ]:
count_plain_title = df_out['generated_text'].str.startswith(
    ("Plain Title", "**Plain Title")
).sum()
count_plain_title
pct = 100 * count_plain_title / len(df_out)
print(f"{count_plain_title} textos ({pct:.2f}%) empiezan con 'Plain Title:'")

19 textos (38.00%) empiezan con 'Plain Title:'


In [ ]:
# ============================================================================
# 📊 PASO 1: CALCULAR MÉTRICAS (BERTScore + TextStat + AlignScore)
# ============================================================================

from bert_score import score as bertscore
import textstat
import pandas as pd
import torch
import transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

EMB_MODEL = "sentence-transformers/all-mpnet-base-v2"  # cambia por modelo biomédico si quieres
DEVICE = "cuda" if (torch and torch.cuda.is_available()) else "cpu"
EMB_BATCH = 256   # reducir si OOM en T4 (prueba 128/64)

'''# Fix para AdamW
if not hasattr(transformers, "AdamW"):
    from torch.optim import AdamW
    transformers.AdamW = AdamW

from alignscore import AlignScore
'''

print("="*80)
print("📊 CALCULANDO MÉTRICAS")
print("="*80)

# --------------------------------------------------------
# 🔧 CONFIGURACIÓN INICIAL
# --------------------------------------------------------
col_ref = "response" #pls
col_gen = "generated_text"  # ← Cambié a generated_text (ajusta si es otro)
col_non_pls="source"

# Crear columna limpia si no existe
if col_gen not in df_out.columns and "generated_text" in df_test.columns:
    df_out["generated_clean"] = df_out["generated_text"].str.strip()
    col_gen = "generated_clean"

print(f"\n✓ Columna de referencia: {col_ref}")
print(f"✓ Columna generada: {col_gen}")
print(f"✓ Total textos: {len(df_out)}")

# --------------------------------------------------------
# 🧩 BERTScore
# --------------------------------------------------------
# --------------------------------------------------------
# 🧩 BERTScore 1 — modelo base (por defecto)
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo base)...")

refs = df_out[col_ref].astype(str).tolist()  # textos de referencia (PLS)
preds = df_out[col_gen].astype(str).tolist()  # textos generados
non_pls = df_out[col_non_pls].astype(str).tolist()  # abstract original

# Primer BERTScore: modelo por defecto ("roberta-large") o idioma "en"
P1, R1, F1_1 = bertscore(preds, refs, lang="en", verbose=False)

df_out["BERT_Precision"] = P1.numpy()
df_out["BERT_Recall"] = R1.numpy()
df_out["BERT_F1"] = F1_1.numpy()

print(f"✅ BERTScore (base) calculado")
print(f"   - Precision media: {P1.mean():.3f}")
print(f"   - Recall media: {R1.mean():.3f}")
print(f"   - F1 media: {F1_1.mean():.3f}")


# --------------------------------------------------------
# 🧩 BERTScore 2 — modelo alternativo o comparación distinta
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo alternativo: Longformer)...")

'''from evaluate import load
bertscore_alt = load("bertscore")

results_alt = bertscore_alt.compute(
    predictions=preds,
    references=refs,
    model_type="allenai/longformer-large-4096-finetuned-triviaqa",
    lang="en"
)


# Convertir los resultados en listas de precisión, recall y F1
P2 = results_alt["precision"]
R2 = results_alt["recall"]
F1_2 = results_alt["f1"]

# Guardar en df_out con nombres distintos
df_out["BERT2_Precision_Longformer"] = P2
df_out["BERT2_Recall_Longformer"] = R2
df_out["BERT2_F1_Longformer"] = F1_2

print(f"✅ BERTScore (Longformer) calculado")
print(f"   - Precision media: {sum(P2)/len(P2):.3f}")
print(f"   - Recall media: {sum(R2)/len(R2):.3f}")
print(f"   - F1 media: {sum(F1_2)/len(F1_2):.3f}")

'''


# --------------------------------------------------------
# 📚 TextStat (legibilidad y complejidad)
# --------------------------------------------------------
print("\n📚 Calculando métricas TextStat...")

def get_textstat_metrics(text):
    try:
        return {
            "flesch_reading_ease": textstat.flesch_reading_ease(text),
            "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),
            "smog_index": textstat.smog_index(text),
            "coleman_liau_index": textstat.coleman_liau_index(text),
            "automated_readability_index": textstat.automated_readability_index(text),
            "dale_chall_readability_score": textstat.dale_chall_readability_score(text),
            "difficult_words": textstat.difficult_words(text),
            "linsear_write_formula": textstat.linsear_write_formula(text),
            "gunning_fog": textstat.gunning_fog(text),
        }
    except:
        return {key: None for key in [
            "flesch_reading_ease", "flesch_kincaid_grade", "smog_index",
            "coleman_liau_index", "automated_readability_index",
            "dale_chall_readability_score", "difficult_words",
            "linsear_write_formula", "gunning_fog"
        ]}

textstat_metrics = df_out[col_gen].apply(get_textstat_metrics)
df_textstat = pd.DataFrame(textstat_metrics.tolist()).add_prefix("TextStat_")

df_out = pd.concat([df_out, df_textstat], axis=1)

print(f"✅ TextStat calculado ({len(df_textstat.columns)} métricas)")

# --------------------------------------------------------
# 🔗 AlignScore o Equivalente Rapido
# --------------------------------------------------------
# -------- SI NO HAY SCORE, CALCULARLO (GPU / batching) --------
print("\n🔗 Calculando AlignScore (versión ligera)...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Usando: {device}")

def compute_pairwise_align(ref_texts, gen_texts, model_name=EMB_MODEL, batch_size=EMB_BATCH, device=device):
    if SentenceTransformer is None:
        raise ImportError("sentence-transformers no está instalado. Instálalo con: pip install sentence-transformers")
    if len(ref_texts) != len(gen_texts):
        raise ValueError("refs y gens deben tener la misma longitud")

    model = SentenceTransformer(model_name, device=device)
    model.max_seq_length = 512

    def encode_list(texts):
        embs = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding", leave=False):
            batch = texts[i:i+batch_size]
            b_emb = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
            b_emb = torch.nn.functional.normalize(b_emb, p=2, dim=1)
            embs.append(b_emb)
        return torch.cat(embs, dim=0)

    refs_emb = encode_list(ref_texts)
    gens_emb = encode_list(gen_texts)
    refs_emb = refs_emb.to(device)
    gens_emb = gens_emb.to(device)

    with torch.no_grad():
        cosines = torch.sum(refs_emb * gens_emb, dim=1).clamp(-1.0, 1.0).cpu().numpy()
    return cosines

# Calcular y guardar
align_scores = compute_pairwise_align(non_pls, preds)
df_out["AlignScore"] = align_scores

print(f"✅ AlignScore calculado (cosine embedding)")
print(f"   - Media: {sum(align_scores)/len(align_scores):.3f}")

print("\n" + "="*80)
print("✅ TODAS LAS MÉTRICAS CALCULADAS")
print("="*80)

# ============================================================================
# 📊 PASO 2: RESUMEN DE MÉTRICAS - TABLA PROFESIONAL
# ============================================================================

from tabulate import tabulate

print("\n🔧 Preparando tablas de resumen...")

# Renombrar TextStat con nombres cortos
textstat_rename = {
    "TextStat_flesch_reading_ease": "FRE",
    "TextStat_flesch_kincaid_grade": "FKGL",
    "TextStat_smog_index": "SMOG",
    "TextStat_coleman_liau_index": "CLI",
    "TextStat_dale_chall_readability_score": "DCRS",
    "TextStat_gunning_fog": "GFI",
}

# Aplicar renombrado
rename_dict = {k: v for k, v in textstat_rename.items() if k in df_out.columns}
df_out_display = df_out.rename(columns=rename_dict)

# Seleccionar columnas de métricas
metric_columns = [
    "BERT_Precision",
    "BERT_Recall",
    "BERT_F1",
    #"BERT2_F1_Longformer",
    "AlignScore"
] + [v for v in textstat_rename.values() if v in df_out_display.columns]

# Añadir columnas TextStat no renombradas
other_textstat = [col for col in df_out.columns if col.startswith("TextStat_") and col not in textstat_rename]
metric_columns.extend(other_textstat)

# Crear DataFrame de métricas sin duplicados
df_metrics = df_out_display[metric_columns].copy()
df_metrics = df_metrics.loc[:, ~df_metrics.columns.duplicated()]

print(f"✓ Métricas seleccionadas: {len(df_metrics.columns)} columnas")

# ============================================================================
# ✅ CREAR stats_df
# ============================================================================

stats_df = pd.DataFrame({
    'Mean': df_metrics.mean(numeric_only=True),
    'Std': df_metrics.std(numeric_only=True),
    'Min': df_metrics.min(numeric_only=True),
    'Max': df_metrics.max(numeric_only=True),
    'Median': df_metrics.median(numeric_only=True),
}).T

print(f"✓ stats_df creado: {stats_df.shape}")

# ============================================================================
# 📈 TABLA DE MÉTRICAS POR TEXTO (primeros 10)
# ============================================================================

print("\n" + "="*120)
print("📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)")
print("="*120)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.3f}'.format)

print("\n")
print(df_metrics.head(10).to_string())
print("\n" + "="*120)

# ============================================================================
# 📊 ESTADÍSTICAS GLOBALES
# ============================================================================

print("\n" + "="*120)
print("📈 ESTADÍSTICAS GLOBALES")
print("="*120 + "\n")

# Preparar datos para tabulate
table_data = []
for stat_name in ['Mean', 'Std', 'Min', 'Max', 'Median']:
    row = [stat_name] + [f"{val:.3f}" for val in stats_df.loc[stat_name]]
    table_data.append(row)

headers = ['Statistic'] + list(stats_df.columns)

print(tabulate(table_data, headers=headers, tablefmt='grid',
               stralign='center', numalign='center'))
print()

# ============================================================================
# 📊 REFERENCIA COMPARATIVA (Modelos del paper CL4Health 2025)
# ============================================================================

reference_data = [
    ["DeepSeek-R1-7B",        16.99, 22.75, 17.69, 12.31, 14.80, 9.45, 0.7955, 0.8496],
    ["Gemini-1.3-Flash",      9.60,  66.87, 8.75,  9.08,  6.90,  5.94, 0.6333, 0.8474],
    ["GPT-4-32K",            12.48,  48.52, 13.39, 11.20, 10.80, 7.41, 0.7801, 0.8519],
    ["GPT-4o",               11.49,  57.13, 11.16,  9.91,  9.09, 6.88, 0.7383, 0.8527],
    ["GPT-3.5-Turbo-16K",    15.52,  28.08, 17.33, 12.59, 13.91, 8.60, 0.8781, 0.8585],
    ["Llama-3.2-3B",         16.42,  21.96, 18.58, 10.79, 15.73, 9.39, 0.8785, 0.8490],
]

ref_headers = ["Model", "CLI ↓", "FRE ↑", "GFI ↓", "SMOG ↓", "FKGL ↓", "DCRS ↓", "AlignScore ↑", "BERTScore ↑"]

print("\n" + "="*120)
print("📊 TABLA DE REFERENCIA (Valores del paper CL4Health-2025)")
print("="*120 + "\n")

print(tabulate(reference_data, headers=ref_headers, tablefmt="grid",
               stralign="center", numalign="center"))
print()

# ============================================================================
# 🎯 RESUMEN EJECUTIVO
# ============================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO")
print("="*100)

key_metrics = {
    "Semantic Similarity": {
        "BERT F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "Flesch Ease (FRE)": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FK Grade Level": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG Index": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "Coleman-Liau": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "Dale-Chall": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "Gunning Fog": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Emoji según valor
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "Flesch" in metric_name or "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "Coleman", "Dale", "Gunning"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            print(f"   {emoji} {metric_name:.<35} {value:>8.3f}")

print("="*100)

# =====================================================================
# 🎯 RESUMEN EJECUTIVO (con comparación a Llama-3.2-3B)
# =====================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)")
print("="*100)

# === 1️⃣ Valores de referencia (Llama-3.2-3B) ===
llama3_ref = {
    "CLI": 16.42,
    "FRE": 21.96,
    "GFI": 18.58,
    "SMOG": 10.79,
    "FKGL": 15.73,
    "DCRS": 9.39,
    "AlignScore": 0.8785,
    "BERT_F1": 0.849
}

# === 2️⃣ Tu resumen de métricas ===
key_metrics = {
    "Semantic Similarity": {
        "BERT_F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "FRE": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FKGL": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "CLI": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "DCRS": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "GFI": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

# === 3️⃣ Mostrar métricas comparadas ===
for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Valor de referencia (si existe)
            ref_val = llama3_ref.get(metric_name, None)

            # Emoji según desempeño relativo
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "CLI", "DCRS", "GFI", "FKGL"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            # Mostrar línea con comparación
            if ref_val is not None:
                diff = value - ref_val
                arrow = "⬆️" if diff > 0 else "⬇️" if diff < 0 else "➡️"
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}  ({arrow} vs Llama3 {ref_val:>8.3f})")
            else:
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}")

print("="*100)
# ============================================================================
# 💾 EXPORTAR A CSV
# ============================================================================

df_metrics.to_csv("metrics_complete.csv", index=False)
stats_df.T.to_csv("metrics_summary.csv")

print("\n💾 Tablas guardadas:")
print("   - metrics_complete.csv (todas las métricas por texto)")
print("   - metrics_summary.csv (estadísticas resumidas)")
print()

📊 CALCULANDO MÉTRICAS

✓ Columna de referencia: response
✓ Columna generada: generated_text
✓ Total textos: 50

📊 Calculando BERTScore (modelo base)...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BERTScore (base) calculado
   - Precision media: 0.869
   - Recall media: 0.834
   - F1 media: 0.851

📊 Calculando BERTScore (modelo alternativo: Longformer)...

📚 Calculando métricas TextStat...
✅ TextStat calculado (9 métricas)

🔗 Calculando AlignScore (versión ligera)...
   Usando: cuda


✅ AlignScore calculado (cosine embedding)
   - Media: 0.862

✅ TODAS LAS MÉTRICAS CALCULADAS

🔧 Preparando tablas de resumen...
✓ Métricas seleccionadas: 13 columnas
✓ stats_df creado: (5, 13)

📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)


   BERT_Precision  BERT_Recall  BERT_F1  AlignScore    FRE   FKGL   SMOG    CLI   DCRS    GFI  TextStat_automated_readability_index  TextStat_difficult_words  TextStat_linsear_write_formula
0           0.839        0.802    0.820       0.491 45.356  9.797 12.215 12.231 10.976 11.959                                10.288                        48                           7.750
1           0.896        0.861    0.878       0.938 31.572 14.023 16.030 14.872 12.032 16.974                                15.917                        66                          15.200
2           0.851        0.826    0.838       0.910 21.117 14.470 15.787 16.852 13.305 17.778                                15.987                        67                          13.167


In [ ]:
import textwrap
import random

# =====================================================================
# EJEMPLO DE SALIDA SIMPLIFICADA
# =====================================================================
print("\n🧾 Ejemplo de salida simplificada:\n" + "="*50)

# Seleccionamos un ejemplo aleatorio del DataFrame
idx = random.randint(0, len(df_out) - 1)

print(f"\n🔹 Índice: {idx}")

# Función auxiliar para mostrar texto con ancho fijo y saltos de línea
def mostrar_texto_formateado(texto, ancho=150):
    # Reemplaza saltos de línea escapados por saltos reales, si los hay
    texto = texto.replace("\\n", "\n")
    # Divide el texto en líneas y aplica textwrap a cada una
    lineas = []
    for linea in texto.splitlines():
        # Ajusta cada línea a máximo 'ancho' caracteres
        lineas.extend(textwrap.wrap(linea, width=ancho, replace_whitespace=False) or [""])
    print("\n".join(lineas))

# Texto original (hasta 1000 caracteres)
print("\n--- Texto original (source) ---")
mostrar_texto_formateado(df_out.loc[idx, "source"][:1000], ancho=150)

# Texto simplificado generado
print("\n--- Texto simplificado generado ---")
mostrar_texto_formateado(df_out.loc[idx, "generated_text"], ancho=150)

print("\n" + "="*50)



🧾 Ejemplo de salida simplificada:

🔹 Índice: 36

--- Texto original (source) ---
Background
Fetal compromise in the term pregnancy is suspected when the following clinical indicators are present: intrauterine growth restriction (IUGR),
decreased fetal movement (DFM), or when investigations such as cardiotocography (CTG) and ultrasound reveal results inconsistent with standard
measurements. Pathological results would necessitate the need for immediate delivery, but the management for ‘suspicious’ results remains unclear and
varies widely across clinical centres. There is clinical uncertainty as to how to best manage women presenting with a suspected term compromised baby
in an otherwise healthy pregnancy.
Objectives
To assess, using the best available evidence, the effects of immediate delivery versus expectant management of the term suspected compromised baby on
neonatal, maternal and long‐term outcomes.
Search methods
We searched the Cochrane Pregnancy and Childbirth Group's Trials R

In [ ]:

import textwrap

# Filtrar los casos con AlignScore < 0.3
low_align = df_out[df_out["AlignScore"] < 0.3]

# Mostrar cuántos hay
print(f"🧮 {len(low_align)} ejemplos con AlignScore < 0.3\n")

# Iterar sobre los primeros 10 ejemplos
for i, row in enumerate(low_align[["source", "response", "generated_text", "AlignScore"]].head(10).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i} | AlignScore: {row.AlignScore:.3f}")
    print("=" * 150)

    # Texto original (source)
    print("\n--- Texto original (source) ---")
    print(textwrap.fill(row.source.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Respuesta de referencia (ground truth)
    print("\n--- Resumen de referencia (response) ---")
    print(textwrap.fill(row.response.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Texto generado por el modelo
    print("\n--- Texto simplificado generado ---")
    print(textwrap.fill(row.generated_text.replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


🧮 0 ejemplos con AlignScore < 0.3



In [ ]:
# Filtrar los casos con AlignScore > 0.8
low_align = df_out[df_out["AlignScore"] > 0.8]

# Mostrar cuántos hay
print(f"🧮 {len(low_align)} ejemplos con AlignScore > 0.8\n")

# Iterar sobre los primeros 10 ejemplos
for i, row in enumerate(low_align[["source", "response", "generated_text", "AlignScore"]].head(10).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i} | AlignScore: {row.AlignScore:.3f}")
    print("=" * 150)

    # Texto original (source)
    print("\n--- Texto original (source) ---")
    print(textwrap.fill(row.source.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Respuesta de referencia (ground truth)
    print("\n--- Resumen de referencia (response) ---")
    print(textwrap.fill(row.response.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Texto generado por el modelo
    print("\n--- Texto simplificado generado ---")
    print(textwrap.fill(row.generated_text.replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


🧮 42 ejemplos con AlignScore > 0.8


🔹 Ejemplo 1 | AlignScore: 0.938

--- Texto original (source) ---
Background
In many countries emergency departments (EDs) are facing an increase in demand for services, long waits, and severe crowding. One response
to mitigate overcrowding has been to provide primary care services alongside or within hospital EDs for patients with non‐urgent problems. However, it
is unknown how this impacts the quality of patient care and the utilisation of hospital resources, or if it is cost‐effective. This is the first
update of the original Cochrane Review published in 2012. 
Objectives
To assess the effects of locating primary care professionals in hospital EDs to
provide care for patients with non‐urgent health problems, compared with care provided by regularly scheduled emergency physicians (EPs). 
Search
methods
We searched the Cochrane Central Register of Controlled Trials (the Cochrane Library; 2017, Issue 4), MEDLINE, Embase, CINAHL, PsycINFO, and
King's 

In [ ]:
for i, text in enumerate(df_out["generated_text"].head(10), start=1):
    print(f"\n🔹 Ejemplo {i}:")
    print(textwrap.fill(text, width=150))  # wrap de 150 caracteres por línea
    print("-" * 150)


🔹 Ejemplo 1:
defining dementia Dementia is a brain disorder that affects memory, thinking, and behavior. It is caused by damage to the brain cells. Dementia can
make it hard to remember things, think clearly, and do everyday tasks. It can affect anyone, but it is most common in older adults. Dementia can be
caused by Alzheimer’s disease, Parkinson’s disease, stroke, or other conditions. Dementia can be mild, moderate, or severe. Mild dementia is when
someone has trouble remembering recent events, but they can still live independently. Moderate dementia is when someone needs help with daily
activities like bathing, dressing, and eating. Severe dementia is when someone cannot recognize family members or friends, and they need constant
supervision. Dementia can be treated with medications, lifestyle changes, and therapies. These treatments can slow down the progression of dementia
and improve quality of life. However, there is no cure for dementia. Rationale Dementia is a serious problem

In [ ]:
!pip install https://github.com/andreasvc/readability/tarball/master

     - 35.0 kB 126.9 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36388 sha256=94b2451c5dc0bdb22d8978dbf5e3e2b543a1e9971c99dab5c8212d5438fa3735
  Stored in directory: /tmp/pip-ephem-wheel-cache-sik752n5/wheels/b9/ca/f5/acdb922de5c90229d7290b12a5b7610a2d44855a0d70b72b48
Successfully built readability


In [ ]:
!pip install syntok bert-score

In [ ]:

print("\n📚 Calculando métricas Readability (PyPI)…")
#from bert_score import score as bertscore
import pandas as pd
import torch
import transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import readability
import syntok.segmenter as segmenter

EMB_MODEL = "sentence-transformers/all-mpnet-base-v2"  # cambia por modelo biomédico si quieres
DEVICE = "cuda" if (torch and torch.cuda.is_available()) else "cpu"
EMB_BATCH = 256   # reducir si OOM en T4 (prueba 128/64)

'''# Fix para AdamW
if not hasattr(transformers, "AdamW"):
    from torch.optim import AdamW
    transformers.AdamW = AdamW

from alignscore import AlignScore
'''

print("="*80)
print("📊 CALCULANDO MÉTRICAS")
print("="*80)

# --------------------------------------------------------
# 🔧 CONFIGURACIÓN INICIAL
# --------------------------------------------------------
col_ref = "response" #pls
col_gen = "generated_text"  # ← Cambié a generated_text (ajusta si es otro)
col_non_pls="source"

# Crear columna limpia si no existe
if col_gen not in df_out.columns and "generated_text" in df_test.columns:
    df_out["generated_clean"] = df_out["generated_text"].str.strip()
    col_gen = "generated_clean"

print(f"\n✓ Columna de referencia: {col_ref}")
print(f"✓ Columna generada: {col_gen}")
print(f"✓ Total textos: {len(df_out)}")




# Lista de métricas que extraeremos
READ_COLS = [
    "FleschReadingEase",
    "FleschKincaidGradeLevel",
    "SmogIndex",
    "GunningFogIndex",
    "ColemanLiauIndex",
    "ARI",
    "DaleChallIndex",
    "LinsearWriteFormula",
    "FryGradeLevel"
]

def preprocess_syntok(text):
    """
    Tokeniza el texto usando syntok EXACTAMENTE
    como requiere readability.getmeasures().
    """
    return '\n\n'.join(
        '\n'.join(
            ' '.join(token.value for token in sentence)
            for sentence in paragraph
        )
        for paragraph in segmenter.analyze(text)
    )

def readability_metrics(text):
    """
    Extrae TODAS las métricas de readability.getmeasures(),
    en un formato fácil de usar.
    """
    try:
        tokenized = preprocess_syntok(text)
        results = readability.getmeasures(tokenized, lang='en')

        grades = results["readability grades"]
        sentences = results["sentence info"]

        out = {}

        # Extraer métricas una por una si existen
        out["FleschReadingEase"] = grades.get("FleschReadingEase")
        out["Kincaid"] = grades.get("Kincaid")
        out["SMOGIndex"] = grades.get("SMOGIndex")
        out["GunningFogIndex"] = grades.get("GunningFogIndex")
        out["Coleman-Liau"] = grades.get("Coleman-Liau")
        out["ARI"] = grades.get("ARI")
        out["DaleChallIndex"] = grades.get("DaleChallIndex")
        out["LinsearWriteFormula"] = grades.get("LinsearWriteFormula")
        out["FryGradeLevel"] = grades.get("FryGradeLevel")

        return out

    except Exception as e:
        # fallo → devolver Nones
        return {name: None for name in READ_COLS}

# Procesar columna col_gen
read_metrics = df_out[col_gen].apply(readability_metrics)
df_read = pd.DataFrame(read_metrics.tolist()).add_prefix("Read_")

df_out = pd.concat([df_out, df_read], axis=1)

print(f"✅ Readability calculado ({len(df_read.columns)} métricas)")




📚 Calculando métricas Readability (PyPI)…
📊 CALCULANDO MÉTRICAS

✓ Columna de referencia: response
✓ Columna generada: generated_text
✓ Total textos: 50
✅ Readability calculado (9 métricas)


In [ ]:
# =====================================================================
# 🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)
# =====================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)")
print("="*100)

# === 1️⃣ Valores de referencia de Llama-3.2-3B ===
llama3_ref = {
    "CLI": 16.42,
    "FRE": 21.96,
    "GFI": 18.58,
    "SMOG": 10.79,
    "FKGL": 15.73,
    "DCRS": 9.39,
    "AlignScore": 0.8785,
    "BERT_F1": 0.8490
}

# === 2️⃣ Métricas de tu modelo ===

key_metrics = {
    "Readability": {
        "FRE": df_read["Read_FleschReadingEase"].mean(),
        "FKGL": df_read["Read_Kincaid"].mean(),
        "SMOG": df_read["Read_SMOGIndex"].mean(),
    },
    "Complexity": {
        "CLI": df_read["Read_Coleman-Liau"].mean(),
        "DCRS": df_read["Read_DaleChallIndex"].mean(),
        "GFI": df_read["Read_GunningFogIndex"].mean(),
    }
}


# === 3️⃣ Comparación clara y txukun ===
for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")

    for metric_name, value in metrics.items():
        if value is None:
            continue

        ref = llama3_ref.get(metric_name, None)

        # símbolo de evaluación relativa
        if ref is not None:
            diff = value - ref
            arrow = "⬆️" if diff > 0 else "⬇️" if diff < 0 else "➡️"
        else:
            diff = None
            arrow = ""

        # emojis de calidad según estándares externos
        if metric_name in ["BERT_F1", "AlignScore"]:
            emoji = "✅" if value >= 0.70 else "⚠️" if value >= 0.50 else "❌"
        elif metric_name == "FRE":
            emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
        else:
            emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"

        # salida final
        if ref is not None:
            print(f"   {emoji} {metric_name:<20} {value:8.3f}   ({arrow} vs Llama-3.2-3B {ref:8.3f})")
        else:
            print(f"   {emoji} {metric_name:<20} {value:8.3f}")



🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)

📌 Readability:
   ❌ FRE                    48.969   (⬆️ vs Llama-3.2-3B   21.960)
   ⚠️ FKGL                   10.575   (⬇️ vs Llama-3.2-3B   15.730)
   ❌ SMOG                   12.161   (⬆️ vs Llama-3.2-3B   10.790)

📌 Complexity:
   ❌ CLI                    13.762   (⬇️ vs Llama-3.2-3B   16.420)
   ✅ DCRS                    6.083   (⬇️ vs Llama-3.2-3B    9.390)
   ❌ GFI                    13.602   (⬇️ vs Llama-3.2-3B   18.580)
